In [2]:
# import pdb;pdb.set_trace();print(__file__)
from distutils.version import StrictVersion
import numpy as np
import os
import itertools
from collections import defaultdict, OrderedDict
import gpustat
import random
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 150)

In [3]:
import hydra
from hydra.experimental import compose, initialize, initialize_config_dir
from omegaconf import OmegaConf, DictConfig
from pprint import pprint

In [4]:
import os
config_dir='/home/jacob/projects/pyleaves/pyleaves/configs'
os.chdir(config_dir)
os.getcwd()

'/home/jacob/projects/pyleaves/pyleaves/configs'

In [5]:
def main(config_dir='/home/jacob/projects/pyleaves/pyleaves/configs/hydra/',
         config_name="PNAS",
         overrides=["+db.user=me"]) -> None:
#     with initialize(config_path=config_path, job_name='test_app'):
    with initialize_config_dir(config_dir=config_dir, job_name='test_app'):
        cfg = compose(config_name=config_name, overrides=overrides)
        pprint(OmegaConf.to_container(cfg))
#     OmegaConf.set_struct(cfg, False)

In [18]:
main(config_name='config', overrides=[])#'vgg6'])

{'checkpoints_path': '???',
 'log_dir': '???',
 'model_dir': '???',
 'n_jobs': 1,
 'num_gpus': 1,
 'pipeline': ['stage_0', 'stage_1'],
 'results_dir': '???',
 'saved_model_path': '???',
 'stage_0': {'dataset': {'color_mode': 'grayscale',
                         'dataset_name': 'Leaves-PNAS',
                         'exclude_classes': ['notcataloged',
                                             'notcatalogued',
                                             'II. IDs, families uncertain',
                                             'Unidentified'],
                         'fold_dir': '/home/jacob/projects/paleoai_data/paleoai_data/v0_2/data/staged_data/Leaves-PNAS/ksplit_10',
                         'include_classes': [],
                         'num_channels': 3,
                         'num_classes': '???',
                         'splits': {'train': 0.5, 'validation': 0.5},
                         'target_size': [512, 512],
                         'threshold': 10},
          

In [3]:
def setGPU():
    stats = gpustat.GPUStatCollection.new_query()
    ids = map(lambda gpu: int(gpu.entry['index']), stats)
    ratios = map(lambda gpu: float(gpu.entry['memory.used'])/float(gpu.entry['memory.total']), stats)
    pairs = list(zip(ids, ratios))
    random.shuffle(pairs)
    bestGPU = min(pairs, key=lambda x: x[1])[0]

    print("setGPU: Setting GPU to: {}".format(bestGPU))
    os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
    os.environ['CUDA_VISIBLE_DEVICES'] = str(bestGPU)

In [4]:
from paleoai_data.utils.kfold_cross_validation import DataFold
from paleoai_data.utils.kfold_cross_validation import generate_KFoldDataset, export_folds_to_csv, KFoldLoader #, prep_dataset
# from pyleaves.utils.multiprocessing_utils import RunAsCUDASubprocess
from pyleaves.utils import multiprocessing_utils
    
fold_dir = '/home/jacob/projects/paleoai_data/paleoai_data/v0_2/data/staged_data/Leaves-PNAS/ksplit_10'
    
kfold_loader = KFoldLoader(root_dir=fold_dir)
kfold_iter = kfold_loader.iter_folds(repeats=1)
# histories = Parallel(n_jobs=n_jobs)(delayed(train_single_fold)(fold=fold, cfg=copy.deepcopy(cfg_0), gpu_device=gpus[i]) for i, fold in enumerate(kfold_iter) if i < n_jobs)

# print(f'Beginning training of models with fold_ids: {fold_ids}')
for i, fold in enumerate(kfold_iter):
    break

In [5]:
# dir(fold)
import copy
from pathlib import Path
import pyleaves
import neptune
import cloudpickle
import itertools
from omegaconf import DictConfig, OmegaConf
# from pyleaves.mains.paleoai_main import restore_or_initialize_experiment, train_single_fold, log_config
from pyleaves.mains.paleoai_main import restore_or_initialize_experiment,log_config
from pyleaves.mains import paleoai_main



CONFIG_DIR = str(Path(pyleaves.RESOURCES_DIR,'..','..','configs','hydra'))
config_path = Path(CONFIG_DIR,'Leaves-PNAS.yaml')
cfg = OmegaConf.load(config_path)
cfg_0 = cfg.stage_0

neptune.init(project_qualified_name=cfg.experiment.neptune_project_name)

Project(jacobarose/sandbox)

In [6]:
# help(OmegaConf.to_container)
# cfg_0_2 = copy.deepcopy(cfg_0)

# worker_id = 0
cfg = restore_or_initialize_experiment(cfg, restore_last=True, prefix='log_dir__', verbose=0)
cfg_0 = copy.deepcopy(cfg.stage_0)

cfg_0.misc['use_tfrecords'] = False

# pickled_func = cloudpickle.dumps(train_single_fold)#(fold, copy.deepcopy(cfg_0), worker_id))
# unpickled_func = cloudpickle.loads(pickled_func)

# worker_id = 0
# pickled_args = cloudpickle.dumps((fold, copy.deepcopy(cfg_0), worker_id))
# unpickled_args = cloudpickle.loads(pickled_args)

# with neptune.create_experiment(name="testing"):
#     unpickled_func(*unpickled_args)

In [7]:
cfg.num_gpus=3
n_jobs = 3

def main():
    histories = []
    args = []
    for worker_id, fold in enumerate(itertools.islice(kfold_iter, n_jobs)):
        if worker_id >= 3:
            break
        args.append((fold, copy.deepcopy(cfg_0), worker_id))
    args = tuple(args)
    with neptune.create_experiment(name=f"testing_worker-{worker_id}"):
        try:
            pool = multiprocessing_utils.RunAsCUDASubprocess(num_gpus=cfg.num_gpus, memory_fraction=0.9)
            result = pool.map(paleoai_main.train_single_fold,n_jobs, *args)
            histories.append(result)
        finally:
            print(f'Finished, worker_id={worker_id}')

                
    return histories


In [ ]:
results = main()

https://ui.neptune.ai/jacobarose/sandbox/e/SAN-763
Beginning n_jobs=3 for 3 total function calls
WORKER 0 INITIATED
WORKER 1 INITIATED
setGPU: GPU:memory ratios initially visible to setGPU:
[(2, 0.0008199409642505739),
 (0, 0.0008199409642505739),
 (1, 0.0008199409642505739),
 (7, 0.0008199409642505739),
 (3, 0.0008199409642505739),
 (6, 0.0008199409642505739),
 (4, 0.0008199409642505739),
 (5, 0.0008199409642505739)]
setGPU: Setting GPU to: 2
WORKER 2 INITIATED
setGPU: GPU:memory ratios initially visible to setGPU:
[(5, 0.0008199409642505739),
 (0, 0.0008199409642505739),
 (4, 0.0008199409642505739),
 (2, 0.0008199409642505739),
 (7, 0.0008199409642505739),
 (3, 0.0008199409642505739),
 (1, 0.0008199409642505739),
 (6, 0.0008199409642505739)]
setGPU: Setting GPU to: 5
setting memory growth failed, continuing anyway.
setGPU: GPU:memory ratios initially visible to setGPU:
[(2, 0.0008199409642505739),
 (3, 0.0008199409642505739),
 (1, 0.0008199409642505739),
 (0, 0.0008199409642505739),


/media/data/conda/jacob/envs/pyleaves2/lib/python3.7/site-packages/scikitplot/plotters.py:37: DeprecationWarning: This module was deprecated in version 0.3.0 and its functions are spread throughout different modules. Please check the documentation and update your function calls as soon as possible. This module will be removed in 0.4.0
  DeprecationWarning)


setting memory growth failed, continuing anyway.
setting memory growth failed, continuing anyway.


/media/data/conda/jacob/envs/pyleaves2/lib/python3.7/site-packages/scikitplot/plotters.py:37: DeprecationWarning: This module was deprecated in version 0.3.0 and its functions are spread throughout different modules. Please check the documentation and update your function calls as soon as possible. This module will be removed in 0.4.0
  DeprecationWarning)
/media/data/conda/jacob/envs/pyleaves2/lib/python3.7/site-packages/scikitplot/plotters.py:37: DeprecationWarning: This module was deprecated in version 0.3.0 and its functions are spread throughout different modules. Please check the documentation and update your function calls as soon as possible. This module will be removed in 0.4.0
  DeprecationWarning)


RUNNING: fold 1 in process None
/media/data/jacob/sandbox_logs/Leaves-PNAS_resnet_50_v2/log_dir__2020-08-19_07-57-30/tfrecord_dir/fold_01
RUNNING: fold 2 in process 1
/media/data/jacob/sandbox_logs/Leaves-PNAS_resnet_50_v2/log_dir__2020-08-19_07-57-30/tfrecord_dir/fold_02
RUNNING: fold 3 in process 2
/media/data/jacob/sandbox_logs/Leaves-PNAS_resnet_50_v2/log_dir__2020-08-19_07-57-30/tfrecord_dir/fold_03
Starting fold 1
inside log dataset
updated cfg
{'checkpoints_path': '/media/data/jacob/sandbox_logs/Leaves-PNAS_resnet_50_v2/log_dir__2020-08-19_07-57-30/model_dir/checkpoints',
 'dataset': {'color_mode': 'grayscale',
             'dataset_name': 'Leaves-PNAS',
             'exclude_classes': ['notcataloged',
                                 'notcatalogued',
                                 'II. IDs, families uncertain',
                                 'Unidentified'],
             'fold_dir': '/home/jacob/projects/paleoai_data/paleoai_data/v0_2/data/staged_data/Leaves-PNAS/ksplit_10'

In [8]:
%debug

> /home/jacob/projects/pyleaves/pyleaves/mains/paleoai_main.py(153)train_single_fold()
    151     K.clear_session()
    152 
--> 153     cfg.tfrecord_dir = os.path.join(cfg.tfrecord_dir,fold.fold_name)
    154     ensure_dir_exists(cfg.tfrecord_dir)
    155     if verbose:



ipdb>  type(cfg


*** SyntaxError: unexpected EOF while parsing


ipdb>  type(cfg)


*** NameError: name 'cfg' is not defined


...

ipdb>  len(args[0])


2


ipdb>  args[0]


self = <pyleaves.utils.multiprocessing_utils.RunAsCUDASubprocess object at 0x7f8b9782c410>
f = <function train_single_fold at 0x7f8b980ef3b0>
n_jobs = 2
args = (((DataFold(fold_id=1)
  19 columns
  19002 train samples
  2112 test samples, {'tfrecord_dir': '/media/data/jacob/sandbox_logs/Leaves-PNAS_resnet_50_v2/log_dir__2020-08-19_07-57-30/tfrecord_dir', 'dataset': {'fold_dir': '/home/jacob/projects/paleoai_data/paleoai_data/v0_2/data/staged_data/Leaves-PNAS/ksplit_10', 'dataset_name': 'Leaves-PNAS', 'num_classes': '???', 'threshold': 10, 'color_mode': 'grayscale', 'num_channels': 3, 'target_size': [512, 512], 'splits': {'train': 0.5, 'validation': 0.5}, 'include_classes': [], 'exclude_classes': ['notcataloged', 'notcatalogued', 'II. IDs, families uncertain', 'Unidentified']}, 'model': {'model_name': 'resnet_50_v2', 'optimizer': 'Adam', 'loss': 'categorical_crossentropy', 'lr': 4e-05, 'regularization': {'l1': 0.0003}, 'METRICS': ['accuracy', 'precision', 'recall']}, 'training': {'batch

ipdb>  q


In [12]:
cfg.num_gpus=2
n_jobs = 2
import pdb

# pool = multiprocessing_utils.RunAsCUDASubprocess(num_gpus=cfg.num_gpus, memory_fraction=0.9)

import asyncio

async def run_fold(*args):
    pool = multiprocessing_utils.RunAsCUDASubprocess(num_gpus=cfg.num_gpus, memory_fraction=0.9)
    return pool(paleoai_main.train_single_fold)(*args)
    



async def main():
    histories = []
    for worker_id, fold in enumerate(itertools.islice(kfold_iter, n_jobs)):
        if worker_id >= 2:
            break
        with neptune.create_experiment(name=f"testing_worker-{worker_id}"):
            try:
#                 result = pool(paleoai_main.train_single_fold)(fold, copy.deepcopy(cfg_0), worker_id)
                print(f'Awaiting, worker_id={worker_id}')
                result = run_fold(fold, copy.deepcopy(cfg_0), worker_id)
                histories.append(result)
            finally:
                print(f'Finished, worker_id={worker_id}')
            continue
                
    return await asyncio.gather(*histories)


In [13]:
main_future = main()

In [ ]:
# main_results = asyncio.run(main_future)
main_results = await main_future

https://ui.neptune.ai/jacobarose/sandbox/e/SAN-753
Awaiting, worker_id=0
Finished, worker_id=0
https://ui.neptune.ai/jacobarose/sandbox/e/SAN-754
Awaiting, worker_id=1
Finished, worker_id=1
LEN(ARGS) = 3
Closed process
WORKER 0 INITIATED
setGPU: GPU:memory ratios initially visible to setGPU:
[(6, 0.0), (5, 0.0), (2, 0.0), (1, 0.0), (3, 0.0), (7, 0.0), (0, 0.0), (4, 0.0)]
setGPU: Setting GPU to: 6
setting memory growth failed, continuing anyway.


/media/data/conda/jacob/envs/pyleaves2/lib/python3.7/site-packages/scikitplot/plotters.py:37: DeprecationWarning: This module was deprecated in version 0.3.0 and its functions are spread throughout different modules. Please check the documentation and update your function calls as soon as possible. This module will be removed in 0.4.0
  DeprecationWarning)


RUNNING: fold 1 in process None
/media/data/jacob/sandbox_logs/Leaves-PNAS_resnet_50_v2/log_dir__2020-08-19_07-57-30/tfrecord_dir/fold_01
creating dataset
Starting fold 1
logging dataset
inside log dataset
updated cfg
cfg
<class 'omegaconf.dictconfig.DictConfig'>
{'checkpoints_path': '/media/data/jacob/sandbox_logs/Leaves-PNAS_resnet_50_v2/log_dir__2020-08-19_07-57-30/model_dir/checkpoints',
 'dataset': {'color_mode': 'grayscale',
             'dataset_name': 'Leaves-PNAS',
             'exclude_classes': ['notcataloged',
                                 'notcatalogued',
                                 'II. IDs, families uncertain',
                                 'Unidentified'],
             'fold_dir': '/home/jacob/projects/paleoai_data/paleoai_data/v0_2/data/staged_data/Leaves-PNAS/ksplit_10',
             'include_classes': [],
             'num_channels': 3,
             'num_classes': 174,
             'splits': {'train': 0.5, 'validation': 0.5},
             'splits_size': {'

In [ ]:
result_0 = main_results[0].get()

In [ ]:
result_0

In [ ]:
%debug

# %debug

histories.append(pool(train_single_fold)
                 (fold, copy.deepcopy(cfg_0), worker_id))
cloudpickle.loads(fn)(*args)

x_col='path'
y_col='family'
index_col='catalog_number'

full_data = fold.full_dataset.data.set_index(index_col)[[x_col, y_col]].rename(columns={x_col:'x', y_col:'y'})

# full_data.index.duplicated(keep=False).sum()
# full_data.describe(include='all')
# full_data[full_data.index.duplicated(keep=False)].sort_index()

full_data

